In [10]:
import os
import pandas as pd
import pandasql as ps
import numpy as np
import glob
from functools import reduce
from pandasql import sqldf
pysqldf = lambda q:sqldf(q, globals())

In [11]:
jan2020 = pd.read_csv('janeiro_2020.csv', usecols = ['cod_consumidor','area', 'tarifario_san','tot_consumo_proc'], low_memory=False)
jan2020.head()
jan2020.shape

(11589, 4)

In [12]:
fev2020=pd.read_csv('fevereiro_2020.csv', usecols = ['cod_consumidor','area', 'tarifario_san','tot_consumo_proc'], low_memory=False)
fev2020.head()
fev2020.shape

(11610, 4)

In [13]:
left_merged=pd.merge(jan2020,fev2020, on=["cod_consumidor","area",'tarifario_san'])
left_merged.head()

,cod_consumidor,area,tarifario_san,tot_consumo_proc_x,tot_consumo_proc_y
0,24068,101,Não doméstico (m3 consumo),3.0,3.0
1,13327,101,Não doméstico (m3 consumo),0.0,0.0
2,25479,101,Não doméstico (m3 consumo),0.0,0.0
3,11533,101,Não doméstico (m3 consumo),10.0,8.0
4,22145,101,Doméstico,8.0,7.0


In [14]:
dez2020=pd.read_csv('dezembro_2020.csv', usecols = ['cod_consumidor','area', 'tipo_consumidor','tarifario_san','tot_consumo_proc'], low_memory=False)
dez2020.head()
dez2020.shape

(11740, 5)

In [15]:
dez2020.to_csv('dez2020.csv', encoding='utf-8')

In [16]:
q="""
    select area,
    count(case when tarifario_san not like 'Não %' or tarifario_san is not null then 1 end) as domesticos_pagantes,
    count(case when tipo_consumidor in ('Doméstico', 'Social - Bombeiros', 'Rio maior 65','Familiar','CE - Rendimento Social Inserção','CE - Complemento Solidário Idosos','CE - Pensão Social  Invalidez','CE - 1.º Escalão Abono Família') and tarifario_san is null then 1 end) as domesticos_n_pagantes, 
    count(case when tarifario_san like 'Não %' then 1 end) as nao_domesticos
    from dez2020
    group by area
    """
ar_consumidores=pysqldf(q)
ar_consumidores.head()

,area,domesticos_pagantes,domesticos_n_pagantes,nao_domesticos
0,101,1189,17,226
1,102,103,1,13
2,103,707,5,84
3,104,360,5,66
4,105,203,21,9


In [179]:
ar_consumidores.to_csv('ar_consumidores_p_area.csv', encoding='utf-8')

In [19]:
files=glob.glob('*.csv')
print('files {}'.format(files))

files ['abril_2020.csv', 'agosto_2020.csv', 'dezembro_2020.csv', 'fevereiro_2020.csv', 'janeiro_2020.csv', 'julho_2020.csv', 'junho_2020.csv', 'maio_2020.csv', 'marco_2020.csv', 'novembro_2020.csv', 'outubro_2020.csv', 'setembro_2020.csv']


In [20]:
df = (pd.read_csv(f, usecols = ['cod_consumidor','area', 'tarifario_san','tot_consumo_proc'], low_memory=False) for f in files)
merge=reduce(lambda left,right: pd.merge(left,right,on=["cod_consumidor","area",'tarifario_san'],how='outer'), df)
merge.head()

,cod_consumidor,area,tarifario_san,tot_consumo_proc_x,tot_consumo_proc_y,tot_consumo_proc_x,tot_consumo_proc_y,tot_consumo_proc_x,tot_consumo_proc_y,tot_consumo_proc_x,tot_consumo_proc_y,tot_consumo_proc_x,tot_consumo_proc_y,tot_consumo_proc_x,tot_consumo_proc_y
0,24068,101,Não doméstico (m3 consumo),7.0,4.0,2.0,3.0,3.0,3.0,4.0,4.0,3.0,2.0,3.0,4.0
1,13327,101,Não doméstico (m3 consumo),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25479,101,Não doméstico (m3 consumo),0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,11533,101,Não doméstico (m3 consumo),9.0,7.0,7.0,8.0,10.0,6.0,10.0,6.0,8.0,15.0,8.0,19.0
4,22145,101,Doméstico,6.0,6.0,5.0,7.0,8.0,6.0,4.0,23.0,7.0,5.0,8.0,5.0


Alterar o nome das colunas e reordenar de acordo com os meses

In [21]:
all_data = merge.set_axis(['cod_consumidor','area','tarifario_san','abril','agosto','dezembro','fevereiro','janeiro','julho','junho','maio','marco','novembro','outubro','setembro'], axis=1, inplace=False)
final_data = all_data[['cod_consumidor','area','tarifario_san','janeiro','fevereiro','marco','abril','maio','junho','julho','agosto','setembro','outubro','novembro','dezembro']]

In [22]:
all_data.head()

,cod_consumidor,area,tarifario_san,abril,agosto,dezembro,fevereiro,janeiro,julho,junho,maio,marco,novembro,outubro,setembro
0,24068,101,Não doméstico (m3 consumo),7.0,4.0,2.0,3.0,3.0,3.0,4.0,4.0,3.0,2.0,3.0,4.0
1,13327,101,Não doméstico (m3 consumo),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25479,101,Não doméstico (m3 consumo),0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,11533,101,Não doméstico (m3 consumo),9.0,7.0,7.0,8.0,10.0,6.0,10.0,6.0,8.0,15.0,8.0,19.0
4,22145,101,Doméstico,6.0,6.0,5.0,7.0,8.0,6.0,4.0,23.0,7.0,5.0,8.0,5.0


In [ ]:
final_data.head(10)

Testes + retirar uma random sample de n= X da df

In [27]:
#final_data.to_csv('file_name.csv', encoding='utf-8')
#final_data.shape

#retirar uma amostra de 70 dados random
#df_elements = final_data.sample(n=70)
#df_elements.to_csv('sample.csv', encoding='utf-8')
#df_elements.head(70)


,cod_consumidor,area,tarifario_san,janeiro,fevereiro,marco,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro
2917,13982,108,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10872,2157,1101,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1625,17611,103,Doméstico,6.0,4.0,4.0,4.0,14.0,5.0,4.0,6.0,4.0,3.0,7.0,5.0
6581,19387,123,Doméstico,19.0,23.0,23.0,25.0,84.0,34.0,33.0,29.0,25.0,22.0,25.0,21.0
6731,23491,124,Doméstico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6070,17045,121,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8160,5333,402,NaN,2.0,1.0,1.0,1.0,4.0,1.0,1.0,2.0,1.0,3.0,0.0,0.0
7500,19805,301,Doméstico,8.0,14.0,14.0,15.0,41.0,34.0,37.0,33.0,15.0,6.0,5.0,6.0
7165,24285,201,Doméstico,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [37]:
q1="""
select area,
sum(case when tarifario_san not like 'Não do%' then janeiro end) as jan_domesticos,
sum(case when tarifario_san like 'Não dom%' then janeiro end) as jan_nao_domesticos,
sum(case when tarifario_san not like 'Não do%' then fevereiro end) as fev_domesticos,
sum(case when tarifario_san like 'Não dom%' then fevereiro end) as fev_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then marco end) as mar_domesticos,
sum(case when tarifario_san like 'Não dom%' then marco end) as mar_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then abril end) as abr_domesticos,
sum(case when tarifario_san like 'Não dom%' then abril end) as abr_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then maio end) as mai_domesticos,
sum(case when tarifario_san like 'Não dom%' then maio end) as mai_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then junho end) as jun_domesticos,
sum(case when tarifario_san like 'Não dom%' then junho end) as jun_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then julho end) as jul_domesticos,
sum(case when tarifario_san like 'Não dom%' then julho end) as jul_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then agosto end) as ago_domesticos,
sum(case when tarifario_san like 'Não dom%' then agosto end) as ago_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then setembro end) as set_domesticos,
sum(case when tarifario_san like 'Não dom%' then setembro end) as set_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then outubro end) as out_domesticos,
sum(case when tarifario_san like 'Não dom%' then outubro end) as out_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then novembro end) as nov_domesticos,
sum(case when tarifario_san like 'Não dom%' then novembro end) as nov_nao_domestico,
sum(case when tarifario_san not like 'Não do%' then dezembro end) as dez_domesticos,
sum(case when tarifario_san like 'Não dom%' then dezembro end) as dez_nao_domestico
from final_data
where tarifario_san is not null
group by area
    """
ar_consumos=pysqldf(q1)
ar_consumos.head()

,area,jan_domesticos,jan_nao_domesticos,fev_domesticos,fev_nao_domestico,mar_domesticos,mar_nao_domestico,abr_domesticos,abr_nao_domestico,mai_domesticos,...,ago_domesticos,ago_nao_domestico,set_domesticos,set_nao_domestico,out_domesticos,out_nao_domestico,nov_domesticos,nov_nao_domestico,dez_domesticos,dez_nao_domestico
0,101,7624.0,12157.0,7141.0,10269.0,7141.0,10269.0,7314.0,10563.0,25737.0,...,9872.0,21558.0,8568.0,13780.0,6991.0,10133.0,7154.0,8046.0,6996.0,7588.0
1,102,539.0,27.0,491.0,20.0,491.0,20.0,500.0,31.0,1968.0,...,875.0,31.0,523.0,17.0,622.0,21.0,561.0,20.0,612.0,17.0
2,103,4247.0,778.0,4024.0,673.0,4024.0,673.0,4359.0,1150.0,13449.0,...,6167.0,1156.0,4475.0,626.0,3974.0,666.0,5067.0,938.0,4293.0,641.0
3,104,1900.0,498.0,1848.0,623.0,1848.0,623.0,1960.0,299.0,6134.0,...,2686.0,749.0,2007.0,556.0,2018.0,499.0,2025.0,514.0,2261.0,533.0
4,105,1093.0,164.0,1027.0,159.0,1027.0,159.0,1119.0,152.0,3761.0,...,1811.0,175.0,1354.0,145.0,1356.0,171.0,1347.0,151.0,1047.0,138.0


In [38]:
ar_consumos.to_csv('ar_consumos.csv', encoding='utf-8')